# <center>Exploratory Data Analysis</center>

## <center>PAKDD2010_Prediction_Data.txt</center>


In [1]:
import pandas as pd
import xlrd
import IPython.display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from src.visualization import visualize

### Load dataset

In [2]:
#take headers from VariablesList.XLS
headers_df = pd.read_excel('../data/raw/PAKDD2010_VariablesList.XLS') # can also index sheet by name or fetch all sheets
headers = headers_df['Var_Title'].tolist()

print(f'{type(headers)}\n'
      f'{len(headers)} features in original list\n'
      f'{len(set(headers))} unique elements in list')

<class 'list'>
54 features in original list
53 unique elements in list


In [3]:
#correct duplicate headers

headers_unique = []

for header in headers:
    if header not in headers_unique:
        headers_unique.append(header)
    else:
        header_duplicated = header + "_2"
        headers_unique.append(header_duplicated)
        print(f'{header} is duplicated and is replaced by {header_duplicated}')

EDUCATION_LEVEL is duplicated and is replaced by EDUCATION_LEVEL_2


In [4]:
#create train dataframe with headers
X_train = pd.read_csv('../data/raw/PAKDD2010_Prediction_Data.txt', sep='\t', encoding= 'unicode_escape', names= headers_unique, low_memory=False, index_col ='ID_CLIENT' )
IPython.display.HTML(X_train.head().to_html())

,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL_2,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
ID_CLIENT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70001,C,10,Web,NaN,1,F,1,0,5,SP,brasileira,1,SP,sao paulo,mooca,Y,5,1.0,29.0,N,0,1118.93,0.0,1,0,0,0,0,1,1,0.0,1,Y,SP,sao paulo,mooca,Y,5,0,9,2.0,NaN,NaN,0,0,0,0,2,N,29,318,318,NaN
70002,C,10,Web,NaN,1,F,1,0,3,SP,FLORA RICA,1,SP,BARRA BONITA,JD NOVA BARRA,Y,12,1.0,8.0,Y,0,600.00,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9,4.0,NaN,2.0,0,0,0,0,2,N,41,173,173,NaN
70003,C,25,Web,NaN,1,F,2,0,4,SP,A PIQUIRI,1,SP,Guarulhos,Cidade Soberana,Y,5,1.0,5.0,Y,0,800.00,0.0,0,0,0,0,0,0,0,0.0,0,Y,SP,Guarulhos,Jardim Santa Vicencia,Y,5,0,9,2.0,NaN,NaN,0,0,0,0,1,N,38,716,716,NaN
70004,C,5,Web,NaN,1,F,1,0,3,PR,MANDAGUARI,1,PR,MANDAGUARI,JARDIM ESPLANDADA,Y,44,1.0,10.0,Y,0,549.00,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9,2.0,NaN,NaN,0,0,0,0,1,N,45,869,869,NaN
70005,C,20,Web,NaN,1,F,1,0,4,RJ,RIO DE JANEIRO,1,RJ,SAQUAREMA,BARREIRA,Y,22,1.0,21.0,N,0,440.00,460.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,7,NaN,NaN,NaN,0,0,0,0,1,N,49,289,289,NaN


In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 70001 to 90000
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CLERK_TYPE                      20000 non-null  object 
 1   PAYMENT_DAY                     20000 non-null  int64  
 2   APPLICATION_SUBMISSION_TYPE     20000 non-null  object 
 3   QUANT_ADDITIONAL_CARDS          1273 non-null   float64
 4   POSTAL_ADDRESS_TYPE             20000 non-null  int64  
 5   SEX                             20000 non-null  object 
 6   MARITAL_STATUS                  20000 non-null  int64  
 7   QUANT_DEPENDANTS                20000 non-null  int64  
 8   EDUCATION_LEVEL                 20000 non-null  int64  
 9   STATE_OF_BIRTH                  20000 non-null  object 
 10  CITY_OF_BIRTH                   19990 non-null  object 
 11  NACIONALITY                     20000 non-null  int64  
 12  RESIDENCIAL_STATE           

In [6]:
print(f'Data without missing values: {X_train.dropna().shape[0]:,} applicants of a total {X_train.shape[0]:,} corresponding a {round((X_train.dropna().shape[0])/(X_train.shape[0]),4)*100}%.')

Data without missing values: 0 applicants of a total 20,000 corresponding a 0.0%.


In [7]:
visualize.missing_values(X_train)

                        NaN Values  % of NaN
TARGET_LABEL_BAD=1           20000    100.00
EDUCATION_LEVEL_2            19610     98.05
MATE_PROFESSION_CODE         18879     94.40
QUANT_ADDITIONAL_CARDS       18727     93.64
PROFESSIONAL_CITY            14344     71.72
PROFESSIONAL_BOROUGH         14344     71.72
MONTHS_IN_RESIDENCE           2524     12.62
OCCUPATION_TYPE               2372     11.86
RESIDENCE_TYPE                 560      2.80
CITY_OF_BIRTH                   10      0.05

                              Empty Values  % of empty
PROFESSIONAL_PHONE_AREA_CODE         15048       75.24
PROFESSIONAL_STATE                   14413       72.06
RESIDENCIAL_PHONE_AREA_CODE           4231       21.16
PROFESSIONAL_BOROUGH                   297        1.48
PROFESSIONAL_CITY                       74        0.37
STATE_OF_BIRTH                          10        0.05
RESIDENCIAL_BOROUGH                      2        0.01


In [8]:
print(f'Dataset features datatypes:\n{X_train.dtypes.value_counts()}')

Dataset features datatypes:
int64      25
object     18
float64    10
dtype: int64


In [9]:
cat_features = X_train.select_dtypes('O').nunique()
cat_features_list = list(cat_features.index)
print(f'Categorical columns ({cat_features.shape[0]}):\n{cat_features}')

Categorical columns (18):
CLERK_TYPE                         1
APPLICATION_SUBMISSION_TYPE        2
SEX                                2
STATE_OF_BIRTH                    29
CITY_OF_BIRTH                   5376
RESIDENCIAL_STATE                 27
RESIDENCIAL_CITY                2599
RESIDENCIAL_BOROUGH             7943
FLAG_RESIDENCIAL_PHONE             2
RESIDENCIAL_PHONE_AREA_CODE       78
FLAG_MOBILE_PHONE                  2
COMPANY                            2
PROFESSIONAL_STATE                28
PROFESSIONAL_CITY               1368
PROFESSIONAL_BOROUGH            2521
FLAG_PROFESSIONAL_PHONE            2
PROFESSIONAL_PHONE_AREA_CODE      69
FLAG_ACSP_RECORD                   2
dtype: int64


In [10]:
cat_num_features = X_train.select_dtypes("number").nunique()[X_train.select_dtypes("number").nunique()<15]
cat_num_features_list = list(cat_num_features.index)
cat_num_features_list.append('PROFESSION_CODE')
cat_num_features_list.append('MATE_PROFESSION_CODE')
print(f'Categorical numerical columns ({X_train[cat_num_features_list].shape[1]}):\n{X_train[cat_num_features_list].nunique()}')

Categorical numerical columns (26):
PAYMENT_DAY                        6
QUANT_ADDITIONAL_CARDS             2
POSTAL_ADDRESS_TYPE                2
MARITAL_STATUS                     7
EDUCATION_LEVEL                    5
NACIONALITY                        3
RESIDENCE_TYPE                     6
FLAG_EMAIL                         2
FLAG_VISA                          2
FLAG_MASTERCARD                    2
FLAG_DINERS                        2
FLAG_AMERICAN_EXPRESS              2
FLAG_OTHER_CARDS                   2
QUANT_BANKING_ACCOUNTS             3
QUANT_SPECIAL_BANKING_ACCOUNTS     3
QUANT_CARS                         2
OCCUPATION_TYPE                    5
EDUCATION_LEVEL_2                  6
FLAG_HOME_ADDRESS_DOCUMENT         2
FLAG_RG                            2
FLAG_CPF                           2
FLAG_INCOME_PROOF                  2
PRODUCT                            3
TARGET_LABEL_BAD=1                 0
PROFESSION_CODE                   20
MATE_PROFESSION_CODE              16
dt

In [11]:
number_dtype = X_train.select_dtypes("number").columns
num_features_list = list(set(number_dtype) - set(cat_num_features.index))
num_features_list.remove('PROFESSION_CODE')
num_features_list.remove('MATE_PROFESSION_CODE')


num_features = X_train.select_dtypes("number").nunique()[num_features_list]
print(f'Numerical columns ({num_features.shape[0]}):\n{num_features}')

Numerical columns (9):
PROFESSIONAL_ZIP_3          750
OTHER_INCOMES               337
MONTHS_IN_RESIDENCE          76
QUANT_DEPENDANTS             15
PERSONAL_MONTHLY_INCOME    4138
RESIDENCIAL_ZIP_3           750
MONTHS_IN_THE_JOB            15
PERSONAL_ASSETS_VALUE       101
AGE                          83
dtype: int64


In [12]:
print(f'Categorical columns ({cat_features.shape[0]}):\n{cat_features}\n\n')
print(f'Categorical numerical columns ({X_train[cat_num_features_list].shape[1]}):\n{X_train[cat_num_features_list].nunique()}\n\n')
print(f'Numerical columns ({num_features.shape[0]}):\n{num_features}')

Categorical columns (18):
CLERK_TYPE                         1
APPLICATION_SUBMISSION_TYPE        2
SEX                                2
STATE_OF_BIRTH                    29
CITY_OF_BIRTH                   5376
RESIDENCIAL_STATE                 27
RESIDENCIAL_CITY                2599
RESIDENCIAL_BOROUGH             7943
FLAG_RESIDENCIAL_PHONE             2
RESIDENCIAL_PHONE_AREA_CODE       78
FLAG_MOBILE_PHONE                  2
COMPANY                            2
PROFESSIONAL_STATE                28
PROFESSIONAL_CITY               1368
PROFESSIONAL_BOROUGH            2521
FLAG_PROFESSIONAL_PHONE            2
PROFESSIONAL_PHONE_AREA_CODE      69
FLAG_ACSP_RECORD                   2
dtype: int64


Categorical numerical columns (26):
PAYMENT_DAY                        6
QUANT_ADDITIONAL_CARDS             2
POSTAL_ADDRESS_TYPE                2
MARITAL_STATUS                     7
EDUCATION_LEVEL                    5
NACIONALITY                        3
RESIDENCE_TYPE                     

In [13]:
visualize.series_count(X_train['TARGET_LABEL_BAD=1'])

,Count,%
NaN,20000,100.0
